In [20]:
import os, shutil
from PIL import Image
from os import listdir
from os.path import isfile, join
import glob

colorDirectory = './color'
greysScaleDirectory = './greyscale'

def listFiles(directory):
    return glob.glob(colorDirectory + '/*jpg')

def removeGreyscale(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
            


print(listFiles(colorDirectory))
removeGreyscale(greysScaleDirectory)
for x in range(0, 3):
    img = Image.open('./color/tomorrow.png').convert('LA')
    img.save('./greyscale/tomorrow' + str(x) + '.png')
    


['./color/war_tank (36).jpg', './color/war_tank (259).jpg', './color/war_tank (609).jpg', './color/war_tank (520).jpg', './color/war_tank (170).jpg', './color/war_tank (465).jpg', './color/war_tank (61).jpg', './color/war_tank (127).jpg', './color/war_tank (577).jpg', './color/war_tank (598).jpg', './color/war_tank (432).jpg', './color/war_tank (98).jpg', './color/war_tank (218).jpg', './color/war_tank (648).jpg', './color/war_tank (561).jpg', './color/war_tank (131).jpg', './color/war_tank (424).jpg', './color/war_tank (20).jpg', './color/war_tank (166).jpg', './color/war_tank (536).jpg', './color/war_tank (473).jpg', './color/war_tank (189).jpg', './color/war_tank (664).jpg', './color/war_tank (234).jpg', './color/war_tank (371).jpg', './color/war_tank (408).jpg', './color/war_tank (263).jpg', './color/war_tank (633).jpg', './color/war_tank (326).jpg', './color/war_tank (625).jpg', './color/war_tank (275).jpg', './color/war_tank (330).jpg', './color/war_tank (449).jpg', './color/war_

FileNotFoundError: [Errno 2] No such file or directory: './color/tomorrow.png'